In [1]:
import pandas as pd

In [23]:
edges = pd.read_csv('data/hetionet-v1.0-edges.tsv', sep='\t', header=0)

In [24]:
nodes = pd.read_csv('data/hetionet-v1.0-nodes.tsv.tsv', sep='\t', header=0)

In [4]:
nodes.head()

,id,name,kind
0,Anatomy::UBERON:0000002,uterine cervix,Anatomy
1,Anatomy::UBERON:0000004,nose,Anatomy
2,Anatomy::UBERON:0000006,islet of Langerhans,Anatomy
3,Anatomy::UBERON:0000007,pituitary gland,Anatomy
4,Anatomy::UBERON:0000010,peripheral nervous system,Anatomy


In [42]:
set(nodes['kind'])

{'Anatomy',
 'Biological Process',
 'Cellular Component',
 'Compound',
 'Disease',
 'Gene',
 'Molecular Function',
 'Pathway',
 'Pharmacologic Class',
 'Side Effect',
 'Symptom'}

In [5]:
edges.head()

,source,metaedge,target
0,Gene::9021,GpBP,Biological Process::GO:0071357
1,Gene::51676,GpBP,Biological Process::GO:0098780
2,Gene::19,GpBP,Biological Process::GO:0055088
3,Gene::3176,GpBP,Biological Process::GO:0010243
4,Gene::3039,GpBP,Biological Process::GO:0006898


# make map to map IDs with actual name and kind

In [8]:
source_or_target_list_map = nodes['id']

In [6]:
"""" function that makes dictionary from two lists"""
def make_dict(list1, list2):
    return dict(zip(list1, list2))

In [14]:
dict_name = make_dict(source_or_target_list_map, nodes['name'])

In [10]:
dict_kind= make_dict(source_or_target_list_map, nodes['kind'])

In [12]:
final_network_map = edges.copy(deep=True)

In [15]:
final_network_map['source_name'] = final_network_map['source'].map(dict_name)
final_network_map['target_name'] = final_network_map['target'].map(dict_name)


In [16]:
final_network_map['kind_of_source'] = final_network_map['source'].map(dict_kind)

In [17]:
final_network_map['kind_of_target'] = final_network_map['target'].map(dict_kind)

In [18]:
final_network_map

,source,metaedge,target,source_name,target_name,kind_of_source,kind_of_target
0,Gene::9021,GpBP,Biological Process::GO:0071357,SOCS3,cellular response to type I interferon,Gene,Biological Process
1,Gene::51676,GpBP,Biological Process::GO:0098780,ASB2,response to mitochondrial depolarisation,Gene,Biological Process
2,Gene::19,GpBP,Biological Process::GO:0055088,ABCA1,lipid homeostasis,Gene,Biological Process
3,Gene::3176,GpBP,Biological Process::GO:0010243,HNMT,response to organonitrogen compound,Gene,Biological Process
4,Gene::3039,GpBP,Biological Process::GO:0006898,HBA1,receptor-mediated endocytosis,Gene,Biological Process
...,...,...,...,...,...,...,...
2250192,Anatomy::UBERON:0000057,AeG,Gene::65009,urethra,NDRG4,Anatomy,Gene
2250193,Anatomy::UBERON:0000474,AeG,Gene::80279,female reproductive system,CDK5RAP3,Anatomy,Gene
2250194,Anatomy::UBERON:0002048,AeG,Gene::1211,lung,CLTA,Anatomy,Gene
2250195,Anatomy::UBERON:0002048,AeG,Gene::8843,lung,HCAR3,Anatomy,Gene


In [19]:
list(set(final_network_map['kind_of_source'])) == list(set(final_network_map['kind_of_target']))

False

In [20]:
def find_diff(list1,list2):
    return (list(set(list1) - set(list2)))

In [34]:
list(set(final_network_map['kind_of_source']))

['Disease', 'Compound', 'Pharmacologic Class', 'Anatomy', 'Gene']

In [36]:
list(set(final_network_map['kind_of_target']))

['Disease',
 'Compound',
 'Anatomy',
 'Molecular Function',
 'Pathway',
 'Side Effect',
 'Cellular Component',
 'Biological Process',
 'Gene',
 'Symptom']

In [21]:
find_diff(list(set(final_network_map['kind_of_source'])), list(set(final_network_map['kind_of_target'])))

['Pharmacologic Class']

In [22]:
meta_edge_encoding = pd.read_csv('data/meta_edges_encoding.tsv', sep='\t', header=0)

In [26]:
set(meta_edge_encoding['abbreviation']) == set(final_network_map['metaedge'])

True

## since the meta_edge_ecoding is same as final_network_map metaedges we can encode them

In [29]:
met_edge_map = make_dict(meta_edge_encoding['abbreviation'], meta_edge_encoding['metaedge'])

In [30]:
final_network_map['metaedge'] = final_network_map['metaedge'].map(met_edge_map)

In [32]:
final_network_map.to_csv('data/final_network_map.csv', index=False)

In [40]:
list(set(final_network_map['metaedge']))

['Disease - localizes - Anatomy',
 'Disease - resembles - Disease',
 'Disease - associates - Gene',
 'Compound - palliates - Disease',
 'Gene - participates - Molecular Function',
 'Compound - resembles - Compound',
 'Gene - participates - Pathway',
 'Compound - upregulates - Gene',
 'Pharmacologic Class - includes - Compound',
 'Compound - downregulates - Gene',
 'Compound - treats - Disease',
 'Disease - upregulates - Gene',
 'Anatomy - upregulates - Gene',
 'Gene - participates - Cellular Component',
 'Gene - participates - Biological Process',
 'Gene - covaries - Gene',
 'Gene - interacts - Gene',
 'Disease - presents - Symptom',
 'Anatomy - expresses - Gene',
 'Compound - binds - Gene',
 'Gene > regulates > Gene',
 'Anatomy - downregulates - Gene',
 'Compound - causes - Side Effect',
 'Disease - downregulates - Gene']

# now we can use this csv to make a streamlit app

In [45]:
final_network_map

,source,metaedge,target,source_name,target_name,kind_of_source,kind_of_target
0,Gene::9021,Gene - participates - Biological Process,Biological Process::GO:0071357,SOCS3,cellular response to type I interferon,Gene,Biological Process
1,Gene::51676,Gene - participates - Biological Process,Biological Process::GO:0098780,ASB2,response to mitochondrial depolarisation,Gene,Biological Process
2,Gene::19,Gene - participates - Biological Process,Biological Process::GO:0055088,ABCA1,lipid homeostasis,Gene,Biological Process
3,Gene::3176,Gene - participates - Biological Process,Biological Process::GO:0010243,HNMT,response to organonitrogen compound,Gene,Biological Process
4,Gene::3039,Gene - participates - Biological Process,Biological Process::GO:0006898,HBA1,receptor-mediated endocytosis,Gene,Biological Process
...,...,...,...,...,...,...,...
2250192,Anatomy::UBERON:0000057,Anatomy - expresses - Gene,Gene::65009,urethra,NDRG4,Anatomy,Gene
2250193,Anatomy::UBERON:0000474,Anatomy - expresses - Gene,Gene::80279,female reproductive system,CDK5RAP3,Anatomy,Gene
2250194,Anatomy::UBERON:0002048,Anatomy - expresses - Gene,Gene::1211,lung,CLTA,Anatomy,Gene
2250195,Anatomy::UBERON:0002048,Anatomy - expresses - Gene,Gene::8843,lung,HCAR3,Anatomy,Gene


In [46]:
list(set(final_network_map['kind_of_source']))

['Disease', 'Compound', 'Pharmacologic Class', 'Anatomy', 'Gene']

In [47]:
list(set(final_network_map['kind_of_target']))

['Disease',
 'Compound',
 'Anatomy',
 'Molecular Function',
 'Pathway',
 'Side Effect',
 'Cellular Component',
 'Biological Process',
 'Gene',
 'Symptom']

In [50]:
final_network_map.to_csv("data/final_network_map" + '.gzip', \
                                           index=False, \
                                           compression='gzip')

In [ ]:
import os

In [51]:
final_network_map_read = !timeit -n5 -r5 -o pd.read_csv("data/final_network_map" + '.gzip', \
                                             compression='gzip')


3.5 s ± 10.4 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [52]:
final_network_map_read

<TimeitResult : 3.5 s ± 10.4 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)>